# SODA Toolchain

![tutorial-flow](imgs/flow-diagram.png)

# High-Level Application Input (TensorFlow)

### Build a model in TensorFlow (Step 1)

In [13]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np
import voxelmorph as vxm

tf.random.set_seed(seed=0)
print(tf.__version__)
from tensorflow import keras

model_path = "/home/gsorrentino/NonRigidReg/models/abdomreg_intra.h5"
model = vxm.networks.VxmDense.load(model_path)



2.13.1


/home/gsorrentino/NonRigidReg/.venv_soda/lib/python3.8/site-packages/voxelmorph/tf/networks.py:124: UserWarning: int_downsize is deprecated, use the int_resolution parameter.
  warnings.warn('int_downsize is deprecated, use the int_resolution parameter.')


In [14]:
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_addons as tfa

class SpatialTransformer(layers.Layer):
    """3D Spatial Transformer using batched 2D warps and static shape enforcement."""
    def call(self, inputs):
        vol, flow = inputs  # vol: [B,D,H,W,C], flow: [B,D,H,W,3]

        # 1. Enforce static (non-zero) shapes to satisfy TOSA requirements
        #    (TOSA dialect expects all dims ≥ 1 and statically known) 
        vol  = tf.ensure_shape(vol,  [None, vol.shape[1], vol.shape[2], vol.shape[3], vol.shape[4]])
        flow = tf.ensure_shape(flow, [None, flow.shape[1], flow.shape[2], flow.shape[3], 3])          

        # 2. Flatten depth dimension into batch: [B,D,H,W,C] → [B*D,H,W,C]
        shape = tf.shape(vol)
        B, D, H, W, C = shape[0], shape[1], shape[2], shape[3], vol.shape[4]
        vol_flat  = tf.reshape(vol,  tf.stack([B * D, H, W, C]))                                     
        flow_flat = tf.reshape(flow, tf.stack([B * D, H, W, C]))                                     

        # 3. Perform a single batched 2D warp via dense_image_warp,
        #    avoiding tf.map_fn loops entirely 
        moved_flat = tfa.image.dense_image_warp(vol_flat, flow_flat[..., :2])

        # 4. Restore original shape: [B*D,H,W,C] → [B,D,H,W,C]
        moved = tf.reshape(moved_flat, tf.stack([B, D, H, W, C]))                                     
        return moved

def conv_block(x, filters, convs=2, kernel_size=3, activation='relu'):
    for _ in range(convs):
        x = layers.Conv3D(filters, kernel_size, padding='same',
                          kernel_initializer='he_normal')(x)
        x = layers.Activation(activation)(x)
    return x

def build_minimal_voxelmorph(inshape,
                             enc_features=(16, 32, 32, 32),
                             dec_features=(32, 32, 32, 32, 32, 16, 16)):
    moving = layers.Input(shape=(*inshape, 1), name='moving')
    fixed  = layers.Input(shape=(*inshape, 1), name='fixed')
    x = layers.Concatenate(axis=-1)([moving, fixed])

    skips = []
    for f in enc_features:
        x = conv_block(x, f)
        skips.append(x)
        x = layers.MaxPool3D(2)(x)

    x = conv_block(x, enc_features[-1] * 2)

    for f, skip in zip(dec_features, reversed(skips)):
        x = layers.UpSampling3D(2)(x)
        x = layers.Concatenate(axis=-1)([x, skip])
        x = conv_block(x, f)

    flow  = layers.Conv3D(3, 3, padding='same', name='flow')(x)
    moved = SpatialTransformer(name='moved')([moving, flow])

    return models.Model(inputs=[moving, fixed],
                        outputs=[moved, flow],
                        name='VoxelmorphMinimalFlatten')

# Instantiate model for a 128³ volume                        
model = build_minimal_voxelmorph((128, 128, 128))
model.summary()


/home/gsorrentino/NonRigidReg/.venv_soda/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Model: "VoxelmorphMinimalFlatten"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 moving (InputLayer)         [(None, 128, 128, 128, 1)]   0         []                            
                                                                                                  
 fixed (InputLayer)          [(None, 128, 128, 128, 1)]   0         []                            
                                                                                                  
 concatenate (Concatenate)   (None, 128, 128, 128, 2)     0         ['moving[0][0]',              
                                                                     'fixed[0][0]']               
                                                                                                  
 conv3d (Conv3D)             (None, 128, 128, 128, 16)    880       ['conca

### Convert model to protobuf

In [15]:
# salva come onnx
def save_model_as_onnx(model, output_path):
    # Convert the model to a concrete function
    full_model = tf.function(lambda x: model(x))
    full_model = full_model.get_concrete_function(
        [tf.TensorSpec(shape=[None, 128, 128, 128, 1], dtype=tf.float32),
         tf.TensorSpec(shape=[None, 128, 128, 128, 1], dtype=tf.float32)]
    )

    # Convert to a frozen graph
    frozen_func = convert_variables_to_constants_v2(full_model)
    frozen_func.graph.as_graph_def()

    # Save the model as ONNX
    tf.saved_model.save(model, output_path)

In [16]:
!mkdir -p output

In [17]:
import os
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

# 2) Salva in SavedModel (opzionale, se serve)
save_path = os.path.join(os.getcwd(), "model/simple/")
tf.saved_model.save(model, save_path) 

# 3) Definisci la tf.function con due input
@tf.function
def infer(moving, fixed):
    return model([moving, fixed])

# 4) Crea il ConcreteFunction con TensorSpec per entrambi gli input
inp0, inp1 = model.inputs

concrete_func = infer.get_concrete_function(
    moving=tf.TensorSpec(shape=inp0.shape, dtype=inp0.dtype, name=inp0.name.split(':')[0]),
    fixed =tf.TensorSpec(shape=inp1.shape, dtype=inp1.dtype, name=inp1.name.split(':')[0])
)

# 5) Congela e salva il grafo
frozen_func = convert_variables_to_constants_v2(concrete_func)
tf.io.write_graph(
    graph_or_graph_def=frozen_func.graph,
    logdir=os.getcwd(),
    name="output/frozen_graph.pbtxt",
    as_text=True
)

# printa le informazioni del frozen graph
#print("Frozen graph:")
#print frozen graph
#with tf.io.gfile.GFile("output/frozen_graph.pbtxt", "r") as f:
#    frozen_graph = f.read()
#    print(frozen_graph)

INFO:tensorflow:Assets written to: /home/gsorrentino/NonRigidReg/soda-opt/docs/tutorials/tensorflow/docker-version/model/simple/assets


INFO:tensorflow:Assets written to: /home/gsorrentino/NonRigidReg/soda-opt/docs/tutorials/tensorflow/docker-version/model/simple/assets
2025-05-06 16:53:23.128842: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2025-05-06 16:53:23.129097: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session


'/home/gsorrentino/NonRigidReg/soda-opt/docs/tutorials/tensorflow/docker-version/output/frozen_graph.pbtxt'

![tutorial-flow](imgs/flow-diagram.png)

In [19]:
# stampa i nomi dei nodi di input nel frozengraph
print("Frozen graph input nodes:")
for node in frozen_func.inputs:
    print(node.name)
    print(node.dtype)
    print(node.shape)

for node in frozen_func.outputs:
    print(node.name)
    print(node.dtype)
    print(node.shape)

Frozen graph input nodes:
moving:0
<dtype: 'float32'>
(None, 128, 128, 128, 1)
fixed:0
<dtype: 'float32'>
(None, 128, 128, 128, 1)
Identity:0
<dtype: 'float32'>
(None, 128, 128, 128, 1)
Identity_1:0
<dtype: 'float32'>
(None, 128, 128, 128, 3)


### Transform protobuf into MLIR (Step 2)




In [21]:
!scripts/protobuf-to-tosa_modified.sh output/frozen_graph.pbtxt output/tosa.mlir

2025-05-06 16:53:54.010000: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Lower MLIR to Linalg on Buffers (Step 3)

In [22]:
!scripts/tosa-to-linalg.sh output/tosa.mlir output/linalg-buffers.mlir

output/tosa.mlir:46:25: error: operation being parsed with an unregistered dialect. If this is intended, please use -allow-unregistered-dialect with the MLIR tool used
    %43 = "tf.MaxPool3D"(%42) {data_format = "NDHWC", device = "", ksize = [1, 2, 2, 2, 1], padding = "VALID", strides = [1, 2, 2, 2, 1]} : (tensor<1x128x128x128x16xf32>) -> tensor<1x64x64x64x16xf32>
                        ^


![tutorial-flow](imgs/flow-diagram.png)

# SODA-OPT: HW/SW Partitioning and Optimizer (Step 4)

## How to use soda.launch?

### Automatic selection of custom accelerator region

Using the pass: `-convert-<abstraction_name>-<operation_name>-to-soda`

Such as: `-convert-linalg-generic-to-soda`

### Manual selection of custom accelerator region

Adding the following lines around any code that will become the accelerator:

```mlir
soda.launch {
  // ...
  // Code to be transformed into an accelerator
  // ...
  soda.terminator
}
```

Run next cell and edit [file](output/01searched-edited.mlir).

In [ ]:
!cp output/linalg-buffers.mlir output/01searched-edited.mlir

# Perform manual edit!

> **⚠️ <span style="color:red;">IMPORTANT:</span> Please modify the file as described below.**

Edit the [file](output/01searched-edited.mlir).

Modify line 101 to the following lines:

```mlir
    soda.launch {
      linalg.batch_matmul ins(%expand_shape_14, %4 : memref<1x4x8xf32>, memref<1x8x4xf32>) outs(%alloc_16 : memref<1x4x4xf32>)
      soda.terminator
    }
```

## Optimization pipeline

![optimizations](imgs/optimization-table.png)

### Kernel without SODA-OPT optimizations (Baseline)

In [ ]:
%%bash
(
  docker run -u $(id -u) -v $(pwd):/working_dir --rm agostini01/soda \
  soda-opt \
    -soda-outline-bambu-code \
    -soda-extract-arguments-to-xml=using-bare-ptr \
    -soda-generate-bambu-accelcode=no-aa \
    -lower-all-to-llvm=use-bare-ptr-memref-call-conv \
    -mlir-print-ir-after-all \
    output/01searched-edited.mlir \
    -o output/04baseline.mlir \
    2>&1 | cat > output/05intermediate-baseline.mlir

  docker run -u $(id -u) -v $(pwd):/working_dir --rm agostini01/soda \
  mlir-translate -opaque-pointers=0  \
    --mlir-to-llvmir \
    output/04baseline.mlir \
    -o output/05baseline.ll
)

Visualize [intermediate file](output/05intermediate-baseline.mlir)

### Kernel with SODA-OPT optimizations

In [ ]:
%%bash
(
  docker run -u $(id -u) -v $(pwd):/working_dir --rm agostini01/soda \
  soda-opt \
    -soda-outline-bambu-code \
    -soda-extract-arguments-to-xml=using-bare-ptr \
    -soda-generate-bambu-accelcode \
    -soda-opt-pipeline-for-bambu=use-bare-ptr-memref-call-conv \
    -mlir-print-ir-after-all \
    output/01searched-edited.mlir \
    -o output/04optimized.mlir \
    2>&1 | cat > output/05intermediate-optimized.mlir

  docker run -u $(id -u) -v $(pwd):/working_dir --rm agostini01/soda \
  mlir-translate -opaque-pointers=0  \
    --mlir-to-llvmir \
    output/04optimized.mlir \
    -o output/05optimized.ll
)


Visualize [intermediate file](output/05intermediate-optimized.mlir)

![tutorial-flow](imgs/flow-diagram.png)

# Bambu: Synthesizing the Outlined Kernel (Step 5)

The following configurations are passed to our backend HLS tool:

* Target: ASIC generation using the Nangate cell library with the FreePDK 45nm kit
* Memory technology: SRAM
* Number of memory channels: 2
  * Supports 2 parallel reads and 2 parallel writes
* Target frequency: 200MHz (5ns period)
* Using bambu's floating-point operation support

You can change parameters passed to bambu in [scripts/run-bambu.sh](scripts/run-bambu.sh)

### Baseline kernel

In [ ]:
! scripts/run-bambu.sh baseline

In [ ]:
baseline_runtime = ""

for runtime in open('output/baseline/bambu-log').readlines():
    if "Average execution" in runtime:
        baseline_runtime = [int(s) for s in runtime.split() if s.isdigit()][0]

print("Average execution in cycles: {}".format(baseline_runtime))

Visualize [Intermediate Dot File](output/baseline/HLS_output/dot/main_kernel/HLS_STGraph.dot)

### Optimized kernel

In [ ]:
! scripts/run-bambu.sh optimized
# Takes aprox 30 seconds to execute

In [ ]:
optimized_runtime = ""

for runtime in open('output/optimized/bambu-log').readlines():
    if "Average execution" in runtime:
        optimized_runtime = [int(s) for s in runtime.split() if s.isdigit()][0]

print("Average execution in cycles: {}".format(optimized_runtime))


Visualize [Intermediate Dot File](output/optimized/HLS_output/dot/main_kernel/HLS_STGraph.dot)

## Comparison of runtime results

* Display runtime
* Display [verilog output file](output/optimized/main_kernel.v)

In [ ]:
print("Average execution in cycles of Baseline kernel:  {}".format(baseline_runtime))
print("Average execution in cycles of Optimized kernel: {}".format(optimized_runtime))
print("Speedup: {:.1f}".format(float(baseline_runtime/optimized_runtime)))

# Commandline interface

To visualize all possible paramenters for our optimization passes run:

- `soda-opt -h`

```
      --soda-opt-pipeline-for-bambu                    
        --affine-tile-size=<ulong>                     
        --bitwidth-of-index-type=<uint>                
        --max-alloc-size-in-bytes=<uint>               
        --max-rank-of-allocated-memref=<uint>          
        --number-of-full-unrolls=<uint>                
        --permutation-map=<uint>                       
        --use-bare-ptr-memref-call-conv                
        --no-alloca-promotion                          
        --no-buffer-trick                              
        --no-scalar-replacement                        
  
```

In [ ]:
%%bash
(
  docker run -u $(id -u) -v $(pwd):/working_dir --rm agostini01/soda \
  soda-opt -h 2>&1 | cat > output/helpfile
)

Open [help file](output/helpfile)

### Modifying the number of unrolls

In [ ]:
%%bash
(
  docker run -u $(id -u) -v $(pwd):/working_dir --rm agostini01/soda \
  soda-opt \
    -soda-outline-bambu-code \
    -soda-extract-arguments-to-xml=using-bare-ptr \
    -soda-generate-bambu-accelcode \
    -soda-opt-pipeline-for-bambu="use-bare-ptr-memref-call-conv number-of-full-unrolls=1" \
    -mlir-print-ir-after-all \
    output/01searched-edited.mlir \
    -o output/04optimized.mlir \
    2>&1 | cat > output/05intermediate-optimized.mlir

  docker run -u $(id -u) -v $(pwd):/working_dir --rm agostini01/soda \
  mlir-translate -opaque-pointers=0  \
    --mlir-to-llvmir \
    output/04optimized.mlir \
    -o output/05optimized.ll
)

Visualize [intermediate file](output/05intermediate-optimized.mlir)

In [ ]:
! scripts/run-bambu.sh optimized

### Default optimization pipeline (again)

Three full unrolls of the inner loop yield better latency for this kernel.

In [ ]:
%%bash
(
  docker run -u $(id -u) -v $(pwd):/working_dir --rm agostini01/soda \
  soda-opt \
    -soda-outline-bambu-code \
    -soda-extract-arguments-to-xml=using-bare-ptr \
    -soda-generate-bambu-accelcode \
    -soda-opt-pipeline-for-bambu=use-bare-ptr-memref-call-conv \
    -mlir-print-ir-after-all \
    output/01searched-edited.mlir \
    -o output/04optimized.mlir \
    2>&1 | cat > output/05intermediate-optimized.mlir

  docker run -u $(id -u) -v $(pwd):/working_dir --rm agostini01/soda \
  mlir-translate -opaque-pointers=0  \
    --mlir-to-llvmir \
    output/04optimized.mlir \
    -o output/05optimized.ll
)

In [ ]:
! scripts/run-bambu.sh optimized

![tutorial-flow](imgs/flow-diagram.png)

# OpenRoad Flow: Automatic ASIC place and route (Step 6)

### Baseline kernel

In [ ]:
! scripts/run-openroad.sh baseline

# Approx. 4min to execute

### Optimized kernel

In [ ]:
! scripts/run-openroad.sh optimized

# Approx. 23min to execute

## Comparison of synthesis results

* Display area
* Display power
* Calculate and display FLOPS/W

In [ ]:
log_path_suffix='HLS_output/Synthesis/bash_flow/openroad/logs/nangate45/main_kernel/base/6_report.log'
gds_path_suffix='HLS_output/Synthesis/bash_flow/openroad/results/nangate45/main_kernel/base/6_final.gds'

### Baseline

In [ ]:
baseline_log='output/baseline/'+log_path_suffix

In [ ]:
power_multiplier = 1 # Open road reports power in W

log_file=baseline_log
total_power = ()

for l in open(log_file, 'r').readlines():
  if ("Total" in l and "Group" not in l):
    total_power=float(l.split()[4])*power_multiplier

  if ("Design area" in l):
    available_area=float(l.split()[2])
    utilization_area=float(l.split()[4].strip('%'))
  

print('Baseline accelerator:')
print('  total power consumption: {}W'.format(total_power))
print('  available chip area: {} um^2'.format(available_area))
print('  utilized chip area: {}%'.format(utilization_area))


baseline_total_power=total_power
baseline_available_area=available_area
baseline_utilization_area=utilization_area


### Optimized for runtime

In [ ]:
optimized_log='output/optimized/'+log_path_suffix

In [ ]:
log_file=optimized_log
total_power = ()

for l in open(log_file, 'r').readlines():
  if ("Total" in l and "Group" not in l):
    total_power=float(l.split()[4])*power_multiplier

  if ("Design area" in l):
    available_area=float(l.split()[2])
    utilization_area=float(l.split()[4].strip('%'))
  

print('Optimized accelerator:')
print('  total power consumption: {}W'.format(total_power))
print('  available chip area: {} um^2'.format(available_area))
print('  utilized chip area: {}%'.format(utilization_area))

optimized_total_power=total_power
optimized_available_area=available_area
optimized_utilization_area=utilization_area

## Post place and route comparison

Considering a matrix multiply kernel has approximatelly 2xNxMxK arithmetic operations

And our selected kernel has the following sizes: 

```mlir
linalg.batch_matmul ins(%A, %B : memref<1x4x8xf32>, memref<1x8x4xf32>) 
                    outs(%C : memref<1x4x4xf32>)

```
M=4, K=8, N=4

We have approximatelly **256** floating point aritihmetic operations

In [ ]:
giga_multiplier=1e9
flop_count = 256 # arithmetic float point operations
target_frequency = 200e+6 # 200MHz

optimized_runtime_in_s = optimized_runtime/target_frequency
baseline_runtime_in_s = baseline_runtime/target_frequency 

baseline_flops_per_watt= flop_count/baseline_runtime_in_s/baseline_total_power
optimized_flops_per_watt= flop_count/optimized_runtime_in_s/optimized_total_power


print("Execution in cycles of Baseline kernel:  {}".format(baseline_runtime))
print("Execution in cycles of Optimized kernel:   {}".format(optimized_runtime))

print("Speedup: \t\t\t{:.2f}x".format(baseline_runtime/optimized_runtime))
print("Area utilization overhead: \t {:.2f}x".format(optimized_utilization_area/baseline_utilization_area))
print("Area overhead: \t\t\t {:.2f}x".format(optimized_available_area/baseline_available_area))
print("Power overhead: \t\t {:.2f}x".format(optimized_total_power/baseline_total_power))

print("Baseline  \t\t\t {:.2f} GFLOPS/W ".format(baseline_flops_per_watt/giga_multiplier))
print("Optimized \t\t\t{:.2f} GFLOPS/W".format(optimized_flops_per_watt/giga_multiplier))


## Generated GDSII files

Output files can be found here:

* output/baseline/HLS_output/Synthesis/bash_flow/openroad/results/nangate45/main_kernel/base/6_final.gds
* output/optimized/HLS_output/Synthesis/bash_flow/openroad/results/nangate45/main_kernel/base/6_final.gds

### Baseline and Optimized Side by Side

![Side-By-Size](imgs/gds-side-by-side.png)

# Thank you!